### Import dependencies

In [2]:
import tensorflow as tf
import os
import numpy as np
import pandas as pd
import pathlib

2023-05-12 19:50:09.408941: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-12 19:50:12.015589: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-12 19:50:12.016815: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-12 19:50:17.942572: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Import data

In [3]:
data_csv =  sorted(str(p) for p in pathlib.Path('data').glob("*.csv"))
data_csv

['data/aapl.csv',
 'data/abbv.csv',
 'data/abt.csv',
 'data/acn.csv',
 'data/adbe.csv',
 'data/amzn.csv',
 'data/avgo.csv',
 'data/bac.csv',
 'data/cmcsa.csv',
 'data/cost.csv',
 'data/crm.csv',
 'data/csco.csv',
 'data/cvx.csv',
 'data/dis.csv',
 'data/googl.csv',
 'data/hd.csv',
 'data/jnj.csv',
 'data/jpm.csv',
 'data/ko.csv',
 'data/lin.csv',
 'data/lly.csv',
 'data/ma.csv',
 'data/mcd.csv',
 'data/meta.csv',
 'data/mrk.csv',
 'data/msft.csv',
 'data/nke.csv',
 'data/nvda.csv',
 'data/pep.csv',
 'data/pfe.csv',
 'data/pg.csv',
 'data/tmo.csv',
 'data/tsla.csv',
 'data/unh.csv',
 'data/v.csv',
 'data/vz.csv',
 'data/wmt.csv',
 'data/xom.csv']

In [4]:
len(data_csv)

38

In [5]:
TrainX = []
TrainY = []
TestX = []
TestY = []
ValidX = []
ValidY = []

In [6]:
period = 20 # days
#TODO: Nomralization
for file in data_csv:
    ds = pd.read_csv(file).iloc[::-1]
    num_row = len(ds)
    num_train = int(num_row*0.70) - 20
    num_valid = int((num_row - num_train)/3)
    num_test = num_row - (num_train + num_valid)
    Y = ds['price']
    X = ds.iloc[:, 1:7]
    # print(X.head())
    for i in range(num_row-20):
        
        if i < num_train:
            TrainY.append(Y.iloc[i+period])
            TrainX.append(X.iloc[i:i+period].to_numpy())
        elif i < num_train + num_test:
            ValidY.append(Y.iloc[i+period])
            ValidX.append(X.iloc[i:i+period].to_numpy())
        else:
            TestY.append(Y.iloc[i+period])
            TestX.append(X.iloc[i:i+period].to_numpy())

#### Get the size of test dataset and confirming correct datatype

In [7]:
print(len(TrainX)+len(ValidX)+len(TestX))
print(TrainX[0].shape)
print(type(TrainX[0]))

200215
(20, 6)
<class 'numpy.ndarray'>


Fun fact: We have more than 200,000 thousand days of data, which is about 548.5 years!!!

#### Converting arrays to Numpy array for the faster computation.

In [8]:
TrainX = np.array(TrainX)
TrainY = np.array(TrainY)
TestX = np.array(TestX)
TestY = np.array(TestY)
ValidX = np.array(ValidX)
ValidY = np.array(ValidY)

#### Save datasets as csv for future use

In [8]:
#TrainX.tofile('./data/TrainX.csv', sep=',')
#TrainY.tofile('./data/TrainY.csv', sep=',')
#TestX.tofile('./data/TestX.csv', sep=',')
#TestY.tofile('./data/TestY.csv', sep=',')
#ValidX.tofile('./data/ValidX.csv', sep=',')
#ValidY.tofile('./data/ValidY.csv', sep=',')

#### Make model

In [12]:
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(120, dropout=0.20, return_sequences=True),
    #tf.keras.layers.Attention(use_scale=False, score_mode='dot'),
    tf.keras.layers.Dense(1)
])

In [13]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='mse')

In [14]:
history = model.fit(x = TrainX, y = TrainY, batch_size=32, epochs=5)

2023-05-12 20:02:45.570175: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 67154400 exceeds 10% of free system memory.


Epoch 1/5


2023-05-12 20:02:46.685879: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-12 20:02:46.688027: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-12 20:02:46.689571: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1902/4373 [============>.................] - ETA: 47s - loss: 1289.5743

KeyboardInterrupt: 